In [16]:
#Importing necessary libraries.
from pyibl import Agent
from random import random
import math
import matplotlib.pyplot as plt
import csv
import pandas as pd
import random
import numpy as np
from pyibl import DelayedResponse
import scipy.stats as stats
import sklearn.metrics
import similarity


df = pd.read_excel("Original_data.xlsx")

# Reset the index of the DataFrame to start from 1
df = df.reset_index(drop=True)
df.index = np.arange(1, len(df) + 1)

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Apply remove_html_tags function to each 'Email' value and store it in a new column 'HTMR'
df['HTMR'] = df['Email'].apply(remove_html_tags)

# Replace 'ham' with 'Ham'
df = df.replace('ham', 'Ham')


ModuleNotFoundError: No module named 'similarity'

In [ ]:
df

In [ ]:
data_at_index_186 = df.loc[187]
print(data_at_index_186)


In [ ]:
#Naming agents.
Alice = Agent(["Decision","Email","Subject","Sender"],default_utility=10.0,noise = 0.25,
              decay = 0.5,
              mismatch_penalty=10.0, 
              default_utility_populates = False)



# Load the similarity data from CSV files
Email_SimVal = pd.read_csv("body_similarity_matrix_bert_non_finetuned")
Subj_SimVal = pd.read_csv("subject_similarity_matrix_bert_non_finetuned")
Sender_SimVal = pd.read_csv("sender_similarity_matrix_bert_non_finetuned")

def limit_to_one(value):
    # Ensure the value is not less than 0
    value = max(value, 0.0)
    # Ensure the value is not greater than 1
    value = min(value, 1.0)
    return value


# Apply the limit_to_one function to all values in the DataFrames
Email_SimVal = Email_SimVal.applymap(limit_to_one)
Subj_SimVal = Subj_SimVal.applymap(limit_to_one)
Sender_SimVal = Sender_SimVal.applymap(limit_to_one)

# Reset index to start from 1
Email_SimVal = Email_SimVal.reset_index(drop=True)
Subj_SimVal = Subj_SimVal.reset_index(drop=True) 
Sender_SimVal = Sender_SimVal.reset_index(drop=True)




# Set row labels to start from 1
Email_SimVal.index = np.arange(1, len(Email_SimVal) + 1)
# Set row labels to start from 1
Subj_SimVal.index = np.arange(1, len(Email_SimVal) + 1)
# Set row labels to start from 1
Sender_SimVal.index = np.arange(1, len(Email_SimVal) + 1)



# Rename columns to start from 1
Email_SimVal.columns = list(range(1, len(Email_SimVal.columns)+1)) 
Subj_SimVal.columns = list(range(1, len(Subj_SimVal.columns)+1))
Sender_SimVal.columns = list(range(1, len(Sender_SimVal.columns)+1))

Alice.similarity(["Email"], lambda x, y: Email_SimVal.iloc[x][y])

Alice.similarity(["Subject"], lambda x, y: Subj_SimVal.iloc[x][y])

Alice.similarity(["Sender"], lambda x, y: Sender_SimVal.iloc[x][y])


# Rewards function for the choice made.
def rewards(choice, index):
  i = index
  if choice == "Phishing":
    if(df["Email_type"][i]== "Phishing"):
      Alice.respond(1.0)
      outcome = 1.0
    else:
      Alice.respond(0.0)
      outcome = 0.0

  else:
    if(df["Email_type"][i] == "Ham" ):
      Alice.respond(1.0)
      outcome = 1.0
    else:
      Alice.respond(0.0)
      outcome = 0.0
  return outcome

In [ ]:
Email_SimVal

In [ ]:
# pre populating function.
def pre_populator(Alice):
  countHam =0
  countPhish = 0
  list_phish = random.sample(range(1,186),4)
  list_ham = random.sample(range(187,239),6)
  list_mix = random.sample((list_ham + list_phish),10)
  # unique randomly. It returns a list of unique items chosen randomly from the list, sequence, or set
    
  for i in list_mix:
    y = np.random.normal(loc=0.5, scale = 0.005) # using normal distribution to decide. mean = 0.5 and stdv = 0.05.
    x = random.random() #for random values used to decide based on human data.
    choice = "Phishing" if x> y else "Ham"
    if df['Email_type'][i] == "Phishing":
      choice = "Phishing" if x < y else "Ham"
      if choice == "Phishing":
        countPhish +=1
      else:
         countHam +=1
    else:
      choice = "Ham" if x < y  else "Phishing"
      if choice == "Phishing":
         countPhish +=1
      else:
         countHam +=1
     # use normal distribution.
    Alice.populate([(choice, i,i,i)], 10.0 if df['Email_type'][i] == choice else -10.0) 
  return countPhish, countHam



In [ ]:
# Rewards function for the choice made.
def rewards(choice, index):
  i = index
  if choice == "Phishing":
    if(df["Email_type"][i]== "Phishing"):
      Alice.respond(1.0)
      outcome = 1.0
    else:
      Alice.respond(0.0)
      outcome = 0.0

  else:
    if(df["Email_type"][i] == "Ham" ):
      Alice.respond(1.0)
      outcome = 1.0
    else:
      Alice.respond(0.0)
      outcome = 0.0
  return outcome

In [ ]:
# Updation to the delayed response.

# Rewards function for the choice made.
def rewards(choice, index):
  i = index
  if choice == "Phishing":
    if(df["Email_type"][i]== "Phishing"):
      Alice.respond(1.0)
      outcome = 1.0
    else:
      Alice.respond(0.0)
      outcome = 0.0

  else:
    if(df["Email_type"][i] == "Ham" ):
      Alice.respond(1.0)
      outcome = 1.0
    else:
      Alice.respond(0.0)
      outcome = 0.0
  return outcome
def updator(list, choice, index):
  updatedOutcome = np.array([0.0]* len(list))
  count = 0
  for i in range(len(list)):
    #total correct
    if(choice[i] == df['Email_type'][index[i]]):
        count +=1

  for i in range(len(list)):
        list[i].update(float(count/10.0))#1.0)#
        updatedOutcome[i] = 1.0 if choice[i] == df["Email_type"][index[i]] else 0

  return updatedOutcome



In [ ]:

#creation of csv file
with open('Model_experiment.csv','w',newline = '') as f:
  fdnames = ['Agent','Participant','Frequency(Base_Rate)','Training Phase','Trail number','Email_ID','ground truth','human_truth', 'choice made/decision', 'outcome','Hit','Miss','False','Correct Rejection']
  writer = csv.DictWriter(f, fieldnames = fdnames)
  writer.writeheader()



In [ ]:
import pandas as pd

# Load data
PreData = pd.read_excel('experiment1.xlsx')

# Data filtering/processing
PreData = PreData[~PreData['email_id'].isin([240,241])]
PreData['email_type'] = PreData['email_type'].replace({'ham': 'Ham', 'phishing': 'Phishing'})
PreData['user_action1'] = PreData['user_action1'].replace({0: 'Ham', 1: 'Phishing'})
PreData["phase"] = pd.to_numeric(PreData["phase"], errors="coerce")

# Sort data
column_order = ["Mturk_id", "phase", "trial"]
PreData = PreData.sort_values(by=column_order)


# Iterate through rows properly with iloc
for p in range(0, len(PreData)):
    
    # Access the full row
    row = PreData.iloc[p] 
    
    # Extract data
    userID = row['Mturk_id']
    userBaseRate = row['base_rate'] 
    userPhase = row['phase']
    userTrial = row['trial']
    userEmailType = row['email_type']
    userEmailID = row['email_id']
    userDecision = row['user_action1']


In [ ]:

#low frequency

num_agents = 98
num_pretrials = 10
Alice.trace = False

# low frequency prepopulation proportion.
low_PhishHamPrepL = np.array([[0]*2]*296)

predata = PreData[(PreData['phase'] == 1) & (PreData['base_rate'] == 25)]
unique_pids1 = predata['Mturk_id'].unique()

postdata = PreData[(PreData['phase'] == 3) & (PreData['base_rate'] == 25)]
unique_pids3 = postdata['Mturk_id'].unique()

tradata = PreData[(PreData['phase'] == 2) & (PreData['base_rate'] == 25)]
unique_pids2 = tradata['Mturk_id'].unique()

for j in range(num_agents):
    curr_pid = unique_pids1[j]
    curr_pid_2 = unique_pids2[j]
    curr_pid_3 = unique_pids3[j]

    # Get rows for participant
    rows1 = predata[predata['Mturk_id'] == curr_pid]
    rows2 = tradata[tradata['Mturk_id'] == curr_pid_2]
    rows3 = postdata[postdata['Mturk_id'] == curr_pid_3] 
    
    emails=list(rows1['email_id'])
    Low_freqE = list(rows2['email_id'])  
    emailP = list(rows3['email_id'])
    
    Alice.reset()
    
    #pre population
    low_PhishHamPrepL[j][0],low_PhishHamPrepL[j][1] = pre_populator(Alice)

    # choice array
    choice1 = np.array([str]*10)
    choice2 = np.array([str]*40)
    choice3 = np.array([str]*10)

    # index array
    index1 = np.array([0]*10)
    index3 = np.array([0]*10)

    # list to store delayed feedbacks of pre and post training phase.
    # list to store blended values
    delayed_Pre = np.array([DelayedResponse]*10)
    outcomePreL = np.array([0.0]*10)
    outcomeTrainL = np.array([0.0]*40)
    outcomePost = np.array([0.0]*10)

    # pretaining low freq
    with open('Model_experiment.csv','a',newline = '') as f:
        writer = csv.DictWriter(f, fieldnames = fdnames)
        for i in range(len(rows1)):
            choicePre = Alice.choose([("Phishing",emails[i],emails[i],emails[i]),("Ham", emails[i],emails[i],emails[i])])
            choice1[i] = choicePre[0]
            index1[i] = choicePre[1]
            delayed_Pre[i] = Alice.respond()
            outcomePreL[i] = 1.0 if choice1[i] == df["Email_type"][emails[i]] else 0.0
            writer.writerow({'Agent':"LoFQA"+str(j+1),
                             'Participant': curr_pid,
                             'Frequency(Base_Rate)': 25,
                             'Training Phase':1,
                             'Trail number': i+1,
                             'Email_ID':emails[i],
                             'ground truth': df["Email_type"][emails[i]],
                             'human_truth': rows1['user_action1'].iloc[i],
                             'choice made/decision': choice1[i],
                             'outcome':1.0 if choice1[i]==df['Email_type'][emails[i]] else 0.0,
                             'Hit':1 if choice1[i]== "Phishing" and df["Email_type"][emails[i]]=="Phishing" else 0,
                             'Miss':1 if choice1[i]== "Ham" and df["Email_type"][emails[i]]=="Phishing" else 0,
                             'False':1 if choice1[i]== "Phishing" and df["Email_type"][emails[i]]=="Ham" else 0,
                             'Correct Rejection':1 if choice1[i]== "Ham" and df["Email_type"][emails[i]]=="Ham" else 0})

        # updating the outcome for pretraining after completion of 10 trials.
        outcomePre = updator(delayed_Pre, choice1,index1)

    # training part for 40 trails.
    with open('Model_experiment.csv','a',newline = '') as f:
        writer = csv.DictWriter(f, fieldnames = fdnames)
        for i in range(len(rows2)):
            choiceTr = Alice.choose([("Phishing",Low_freqE[i],Low_freqE[i],Low_freqE[i]),("Ham", Low_freqE[i],Low_freqE[i],Low_freqE[i])])
            choice2[i] = choiceTr[0]
            outcomeTrainL[i] = rewards(choice2[i],Low_freqE[i])
            writer.writerow({'Agent':"LoFQA"+str(j+1),
                             'Participant': curr_pid_2,
                             'Frequency(Base_Rate)': 25,
                             'Training Phase':2,
                             'Trail number': i+1,
                             'Email_ID':Low_freqE[i],
                             'ground truth': df["Email_type"][Low_freqE[i]],
                             'human_truth': rows2['user_action1'].iloc[i],
                             'choice made/decision': choice2[i], 
                             'outcome':1.0 if choice2[i]==df['Email_type'][Low_freqE[i]] else 0.0,
                             'Hit':1 if choice2[i]== "Phishing" and df["Email_type"][Low_freqE[i]]=="Phishing" else 0,
                             'Miss':1 if choice2[i]== "Ham" and df["Email_type"][Low_freqE[i]]=="Phishing" else 0,
                             'False':1 if choice2[i]== "Phishing" and df["Email_type"][Low_freqE[i]]=="Ham" else 0,
                             'Correct Rejection':1 if choice2[i]== "Ham" and df["Email_type"][Low_freqE[i]]=="Ham" else 0})


    
    with open('Model_experiment.csv','a',newline = '') as f:
        writer = csv.DictWriter(f,fieldnames=fdnames)
        for i in range(len(rows3)):
            choicePost = Alice.choose([("Phishing",emailP[i],emailP[i],emailP[i]),("Ham", emailP[i],emailP[i],emailP[i])])
            choice3[i] = choicePost[0]
            index3[i] = choicePost[1]
            outcomePost[i] = 1.0 if choice3[i] == df["Email_type"][emailP[i]] else 0.0
            Alice.respond()
            writer.writerow({'Agent':"LoFQA"+str(j+1),
                             'Participant': curr_pid_3,
                             'Frequency(Base_Rate)': 25,
                             'Training Phase':3,
                             'Trail number': i+1,
                             'Email_ID':emailP[i],
                             'ground truth': df["Email_type"][emailP[i]],
                             'human_truth': rows3['user_action1'].iloc[i],
                             'choice made/decision': choice3[i],
                             'outcome':1.0 if choice3[i]==df['Email_type'][emailP[i]] else 0.0,
                             'Hit':1 if choice3[i]== "Phishing" and df["Email_type"][emailP[i]]=="Phishing" else 0,
                             'Miss':1 if choice3[i]== "Ham" and df["Email_type"][emailP[i]]=="Phishing" else 0,
                             'False':1 if choice3[i]== "Phishing" and df["Email_type"][emailP[i]]=="Ham" else 0,
                             'Correct Rejection':1 if choice3[i]== "Ham" and df["Email_type"][emailP[i]]=="Ham" else 0})


In [ ]:
 df["Email_type"][186]

In [ ]:
# medium frequency email samples.

num_agents = 99
num_pretrials = 10
Alice.trace = False
med_PhishHamPrepL = np.array([[0]*2]*296)

predata = PreData[(PreData['phase'] == 1) & (PreData['base_rate'] == 50)]
unique_pids1 = predata['Mturk_id'].unique()

postdata = PreData[(PreData['phase'] == 3) & (PreData['base_rate'] == 50)]
unique_pids3 = postdata['Mturk_id'].unique()

tradata = PreData[(PreData['phase'] == 2) & (PreData['base_rate'] == 50)]
unique_pids2 = tradata['Mturk_id'].unique()

for j in range(num_agents):
    curr_pid = unique_pids1[j]
    curr_pid_2 = unique_pids2[j]
    curr_pid_3 = unique_pids3[j]

    # Get rows for participant
    rows1 = predata[predata['Mturk_id'] == curr_pid]
    rows2 = tradata[tradata['Mturk_id'] == curr_pid_2]
    rows3 = postdata[postdata['Mturk_id'] == curr_pid_3] 
    
    emails = list(rows1['email_id'])
    mid_freqE = list(rows2['email_id'])  
    emailP = list(rows3['email_id'])
    
    Alice.reset()
    
    #pre population
    med_PhishHamPrepL[j][0], med_PhishHamPrepL[j][1] = pre_populator(Alice)

    choice1 = np.array([str]*10)
    choice2 = np.array([str] * 40)
    choice3 = np.array([str] * 10)

    index1 = np.array([0] * 10)
    index3 = np.array([0] * 10)

    delayed_Pre = np.array([DelayedResponse] * 10)
    outcomeTrainM = np.array([0.0] * 40)
    outcomePost = np.array([0.0] * 10)
    outcomePre = np.array([0.0] * 10)

    with open('Model_experiment.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fdnames)
        for i in range(len(rows1)):
            choicePre = Alice.choose([("Phishing", emails[i], emails[i], emails[i]), ("Ham", emails[i], emails[i], emails[i])])
            choice1[i] = choicePre[0]
            index1[i] = choicePre[1]
            delayed_Pre[i] = Alice.respond()
            outcomePreL[i] = 1.0 if choice1[i] == df["Email_type"][emails[i]] else 0.0
            writer.writerow({'Agent': "MeFQA" + str(j + 1),
                             'Participant': curr_pid,
                             'Frequency(Base_Rate)': 50,
                             'Training Phase': 1,
                             'Trail number': i + 1,
                             'Email_ID': emails[i],
                             'ground truth': df["Email_type"][emails[i]],
                             'human_truth': rows1['user_action1'].iloc[i],
                             'choice made/decision': choice1[i],
                             'outcome': 1.0 if choice1[i] == df['Email_type'][emails[i]] else 0.0,
                             'Hit': 1 if choice1[i] == "Phishing" and df["Email_type"][emails[i]] == "Phishing" else 0,
                             'Miss': 1 if choice1[i] == "Ham" and df["Email_type"][emails[i]] == "Phishing" else 0,
                             'False': 1 if choice1[i] == "Phishing" and df["Email_type"][emails[i]] == "Ham" else 0,
                             'Correct Rejection': 1 if choice1[i] == "Ham" and df["Email_type"][emails[i]] == "Ham" else 0})

        outcomePre = updator(delayed_Pre, choice1, index1)

    # training part for 40 trails.
    with open('Model_experiment.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fdnames)
        for i in range(len(rows2)):
            choice = Alice.choose([("Phishing", mid_freqE[i], mid_freqE[i], mid_freqE[i]), ("Ham", mid_freqE[i], mid_freqE[i], mid_freqE[i])])
            choice2[i] = choice[0]
            outcomeTrainM[i] = rewards(choice2[i], mid_freqE[i])
            writer.writerow({'Agent': "MeFQA" + str(j + 1),
                             'Participant': curr_pid_2,
                             'Frequency(Base_Rate)': 50,
                             'Training Phase': 2,
                             'Trail number': i + 1,
                             'Email_ID': mid_freqE[i],
                             'ground truth': df["Email_type"][mid_freqE[i]],
                              'human_truth': rows2['user_action1'].iloc[i],
                             'choice made/decision': choice2[i],
                             'outcome': 1.0 if choice2[i] == df['Email_type'][mid_freqE[i]] else 0.0,
                             'Hit': 1 if choice2[i] == "Phishing" and df["Email_type"][mid_freqE[i]] == "Phishing" else 0,
                             'Miss': 1 if choice2[i] == "Ham" and df["Email_type"][mid_freqE[i]] == "Phishing" else 0,
                             'False': 1 if choice2[i] == "Phishing" and df["Email_type"][mid_freqE[i]] == "Ham" else 0,
                             'Correct Rejection': 1 if choice2[i] == "Ham" and df["Email_type"][mid_freqE[i]] == "Ham" else 0})

    # post_training
    with open('Model_experiment.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fdnames)
        for i in range(len(rows3)):
            choicePost = Alice.choose([("Phishing", emailP[i], emailP[i], emailP[i]), ("Ham", emailP[i], emailP[i], emailP[i])])
            choice3[i] = choicePost[0]
            index3[i] = choicePost[1]
            outcomePost[i] = 1.0 if choice3[i] == df["Email_type"][emailP[i]] else 0.0
            Alice.respond()
            writer.writerow({'Agent': "MeFQA" + str(j + 1),
                             'Participant': curr_pid_3,
                             'Frequency(Base_Rate)': 50,
                             'Training Phase': 3,
                             'Trail number': i + 1,
                             'Email_ID': emailP[i],
                             'ground truth': df["Email_type"][emailP[i]],
                              'human_truth': rows3['user_action1'].iloc[i],
                             'choice made/decision': choice3[i],
                             'outcome': 1.0 if choice3[i] == df['Email_type'][emailP[i]] else 0.0,
                             'Hit': 1 if choice3[i] == "Phishing" and df["Email_type"][emailP[i]] == "Phishing" else 0,
                             'Miss': 1 if choice3[i] == "Ham" and df["Email_type"][emailP[i]] == "Phishing" else 0,
                             'False': 1 if choice3[i] == "Phishing" and df["Email_type"][emailP[i]] == "Ham" else 0,
                             'Correct Rejection': 1 if choice3[i] == "Ham" and df["Email_type"][emailP[i]] == "Ham" else 0})


In [ ]:
# high frequency

num_agents = 99
num_pretrials = 10
Alice.trace = False
high_PhishHamPrepL = np.array([[0]*2]*296)

predata = PreData[(PreData['phase'] == 1) & (PreData['base_rate'] == 75)]
unique_pids1 = predata['Mturk_id'].unique()

postdata = PreData[(PreData['phase'] == 3) & (PreData['base_rate'] == 75)]
unique_pids3 = postdata['Mturk_id'].unique()

tradata = PreData[(PreData['phase'] == 2) & (PreData['base_rate'] == 75)]
unique_pids2 = tradata['Mturk_id'].unique()

for j in range(num_agents):  
    curr_pid = unique_pids1[j]
    curr_pid_2 = unique_pids2[j]
    curr_pid_3 = unique_pids3[j]

    # Get rows for participant
    rows1 = predata[predata['Mturk_id'] == curr_pid]
    rows2 = tradata[tradata['Mturk_id'] == curr_pid_2]
    rows3 = postdata[postdata['Mturk_id'] == curr_pid_3]
    
    emails = list(rows1['email_id'])
    high_freqE = list(rows2['email_id'])  
    emailP = list(rows3['email_id'])  
    Alice.reset()

    # choice array
    choice1 = np.array([str]*10)
    choice2 = np.array([str] * 40)
    choice3 = np.array([str] * 10)

    # index array
    index1 = np.array([0] * 10)
    index3 = np.array([0] * 10)

    # List to store delayed feedbacks of pre and post training phase.
    delayed_Pre = np.array([DelayedResponse] * 10)
    outcomeTrainH = np.array([0.0] * 40)
    outcomePost = np.array([0.0] * 10)
    outcomePre = np.array([0.0] * 10)

    # resetting agent every after the inner loops are fully iterated.
    Alice.reset()

    # pre-population
    high_PhishHamPrepL[j][0], high_PhishHamPrepL[j][1] = pre_populator(Alice)

    # choice array
    choice1 = np.array([str] * 10)
    choice2 = np.array([str] * 40)
    choice3 = np.array([str] * 10)

    # index array
    index1 = np.array([0] * 10)
    index3 = np.array([0] * 10)

    # list to store delayed feedbacks of pre and post training phase.
    # list to store blended values
    delayed_Pre = np.array([DelayedResponse] * 10)
    outcomePre = np.array([0.0] * 10)
    outcomeTrainH = np.array([0.0] * 40)
    outcomePost = np.array([0.0] * 10)

    with open('Model_experiment.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fdnames)
        for i in range(len(rows1)):
            choicePre = Alice.choose([("Phishing", emails[i], emails[i], emails[i]), ("Ham", emails[i], emails[i], emails[i])])
            choice1[i] = choicePre[0]
            index1[i] = choicePre[1]
            delayed_Pre[i] = Alice.respond()
            outcomePreL[i] = 1.0 if choice1[i] == df["Email_type"][emails[i]] else 0.0
            writer.writerow({'Agent': "HiFQA" + str(j+1),
                             'Participant': curr_pid,
                             'Frequency(Base_Rate)': 75,
                             'Training Phase': 1,
                             'Trail number': i+1,
                             'Email_ID': emails[i],
                             'ground truth': df["Email_type"][emails[i]],
                              'human_truth': rows1['user_action1'].iloc[i],
                             'choice made/decision': choice1[i], 
                             'outcome': 1.0 if choice1[i] == df['Email_type'][emails[i]] else 0.0,
                             'Hit': 1 if choice1[i] == "Phishing" and df["Email_type"][emails[i]] == "Phishing" else 0,
                             'Miss': 1 if choice1[i] == "Ham" and df["Email_type"][emails[i]] == "Phishing" else 0,
                             'False': 1 if choice1[i] == "Phishing" and df["Email_type"][emails[i]] == "Ham" else 0,
                             'Correct Rejection': 1 if choice1[i] == "Ham" and df["Email_type"][emails[i]] == "Ham" else 0})

        # updating the outcome for pretraining after completion of 10 trials.
        outcomePre = updator(delayed_Pre, choice1, index1)

    # training part for 40 trials.
    with open('Model_experiment.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fdnames)
        for i in range(len(rows2)):
            choice = Alice.choose([("Phishing", high_freqE[i], high_freqE[i], high_freqE[i]), ("Ham", high_freqE[i], high_freqE[i], high_freqE[i])])
            choice2[i] = choice[0]
            outcomeTrainH[i] = rewards(choice2[i], high_freqE[i])
            writer.writerow({'Agent': "HiFQA" + str(j+1),
                             'Participant': curr_pid_2,
                             'Frequency(Base_Rate)': 75,
                             'Training Phase': 2,
                             'Trail number': i+1,
                             'Email_ID': high_freqE[i],
                             'ground truth': df["Email_type"][high_freqE[i]],
                              'human_truth': rows2['user_action1'].iloc[i],
                             'choice made/decision': choice2[i],
                             'outcome': 1.0 if choice2[i] == df['Email_type'][high_freqE[i]] else 0.0,
                             'Hit': 1 if choice2[i] == "Phishing" and df["Email_type"][high_freqE[i]] == "Phishing" else 0,
                             'Miss': 1 if choice2[i] == "Ham" and df["Email_type"][high_freqE[i]] == "Phishing" else 0,
                             'False': 1 if choice2[i] == "Phishing" and df["Email_type"][high_freqE[i]] == "Ham" else 0,
                             'Correct Rejection': 1 if choice2[i] == "Ham" and df["Email_type"][high_freqE[i]] == "Ham" else 0})

    # post_training
    
    with open('Model_experiment.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fdnames)
        for i in range(len(rows3)):
            choicePost = Alice.choose([("Phishing", emailP[i], emailP[i], emailP[i]), ("Ham", emailP[i], emailP[i], emailP[i])])
            choice3[i] = choicePost[0]
            index3[i] = choicePost[1]
            outcomePost[i] = 1.0 if choice3[i] == df["Email_type"][emailP[i]] else 0.0
            Alice.respond()
            writer.writerow({'Agent': "HiFQA" + str(j+1),
                             'Participant': curr_pid_3,
                             'Frequency(Base_Rate)': 75,
                             'Training Phase': 3,
                             'Trail number': i+1,
                             'Email_ID': emailP[i],
                             'ground truth': df["Email_type"][emailP[i]],
                              'human_truth': rows3['user_action1'].iloc[i],
                             'choice made/decision': choice3[i],
                             'outcome': 1.0 if choice3[i] == df['Email_type'][emailP[i]] else 0.0,
                             'Hit': 1 if choice3[i] == "Phishing" and df["Email_type"][emailP[i]] == "Phishing" else 0,
                             'Miss': 1 if choice3[i] == "Ham" and df["Email_type"][emailP[i]] == "Phishing" else 0,
                             'False': 1 if choice3[i] == "Phishing" and df["Email_type"][emailP[i]] == "Ham" else 0,
                             'Correct Rejection': 1 if choice3[i] == "Ham" and df["Email_type"][emailP[i]] == "Ham" else 0})


In [ ]:
df_DataModel = pd.read_csv('Model_experiment.csv')

In [ ]:
df_DataModel

In [ ]:

data_186 = data_model.loc[data_model['Email_ID'] == 186]
print(data_186)


In [ ]:
a=df_DataModel['Agent'].unique()
len(a)

In [ ]:
import pandas as pd
import numpy as np

# Read the model data CSV
data_model = pd.read_csv('Model_experiment.csv')  # Replace 'model_data.csv' with the actual filename

def calculate_metrics_model(participant_data):
    # Calculate hit, false, correct rejection, and miss counts for model data
    hit = participant_data[(participant_data['ground truth'] == 'Phishing') & (participant_data['choice made/decision'] == 'Phishing')].shape[0]
    miss = participant_data[(participant_data['ground truth'] == 'Phishing') & (participant_data['choice made/decision'] == 'Ham')].shape[0]
    correct_rejection = participant_data[(participant_data['ground truth'] == 'Ham') & (participant_data['choice made/decision'] == 'Ham')].shape[0]
    false_alarm = participant_data[(participant_data['ground truth'] == 'Ham') & (participant_data['choice made/decision'] == 'Phishing')].shape[0]

    return hit, false_alarm, correct_rejection, miss

# Create lists to store results for model data
participants_model = []
phases_model = []
base_rates_model = []
hits_model = []
false_alarms_model = []
correct_rejections_model = []
misses_model = []
participant_ids_model = []  # New list to store participant IDs

# Iterate over participants and phases for model data
for agent in data_model['Agent'].unique():
    for phase in data_model['Training Phase'].unique():
        # Filter data for the specified agent and phase
        participant_data_model = data_model[(data_model['Agent'] == agent) & (data_model['Training Phase'] == phase)]

        # Calculate metrics for the current agent and phase
        hit_model, false_alarm_model, correct_rejection_model, miss_model = calculate_metrics_model(participant_data_model)

        # Get the base rate for the current agent and phase
        base_rate_model = participant_data_model['Frequency(Base_Rate)'].iloc[0]

        participants_model.append(agent)
        phases_model.append(phase)
        base_rates_model.append(base_rate_model)
        hits_model.append(hit_model)
        false_alarms_model.append(false_alarm_model)
        correct_rejections_model.append(correct_rejection_model)
        misses_model.append(miss_model)
        
        participant_ids_model.append(participant_data_model['Participant'].iloc[0])  # Append participant ID

# Create a DataFrame with the results for model data
results_df_model = pd.DataFrame({
    'Participant': participant_ids_model,  # Include Participant ID column
    'Agent': participants_model,
    'Phase': phases_model,
    'Base Rate': base_rates_model,
    'Hit': hits_model,
    'False': false_alarms_model,
    'Correct Rejection': correct_rejections_model,
    'Miss': misses_model
})

# Calculate hit rate, false alarm rate, dprime, and bias for model data
results_df_model['HitRate'] = results_df_model['Hit'] / (results_df_model['Hit'] + results_df_model['Miss'])
results_df_model['FARate'] = results_df_model['False'] / (results_df_model['False'] + results_df_model['Correct Rejection'])

# Replace 1.0 and 0.0 values
results_df_model['HitRate'] = np.where(results_df_model['HitRate'] == 1.0, 0.999, results_df_model['HitRate'])
results_df_model['FARate'] = np.where(results_df_model['FARate'] == 1.0, 0.999, results_df_model['FARate'])
results_df_model['HitRate'] = np.where(results_df_model['HitRate'] == 0.0, 0.001, results_df_model['HitRate'])
results_df_model['FARate'] = np.where(results_df_model['FARate'] == 0.0, 0.001, results_df_model['FARate'])

# Calculate dprime and bias for model data
results_df_model['zHR'] = (results_df_model['HitRate'] - results_df_model.groupby(['Base Rate','Phase'])['HitRate'].transform('mean')) / results_df_model.groupby(['Base Rate', 'Phase'])['HitRate'].transform('std')
results_df_model['zFAR'] = (results_df_model['FARate'] - results_df_model.groupby(['Base Rate','Phase'])['FARate'].transform('mean')) / results_df_model.groupby(['Base Rate', 'Phase'])['FARate'].transform('std')
#phase and basse rate 
results_df_model['dprime'] = results_df_model['zHR'] - results_df_model['zFAR']
results_df_model['Bias'] = -(results_df_model['zHR'] + results_df_model['zFAR']) / 2

# Save the results for model data to a CSV file
results_df_model.to_csv('Model.csv', index=False)


In [ ]:
# Calculate overall accuracy
overall_accuracy = (results_df_model['Hit'] + results_df_model['Correct Rejection']).sum() / (results_df_model['Hit'] + results_df_model['Correct Rejection'] + results_df_model['False'] + results_df_model['Miss']).sum()
overall_accuracy

In [ ]:
b=pd.read_csv("Model.csv")

In [ ]:
b

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the human experiment data from the CSV file
results_df = pd.read_csv('Model.csv')

# Define the order of frequencies
frequency_order = [25, 50, 75]


results_df['Base Rate'] = pd.Categorical(results_df['Base Rate'], categories=frequency_order, ordered=True)

# Group by frequency and phase, avg hit rates>
mean_hit_rates = results_df.groupby(['Base Rate', 'Phase'])['HitRate'].mean().reset_index()

pivot_mean_hit_rates = mean_hit_rates.pivot_table(index='Base Rate', columns='Phase', values='HitRate')

# Plotting
pivot_mean_hit_rates.plot(kind='bar', figsize=(12, 8), colormap='viridis')
plt.title('Average Hit Rates for Each Frequency and Phase')
plt.xlabel('Frequency(Base_Rate)')
plt.ylabel('Average Hit Rate')
plt.legend(title='Phase', loc='upper right', labels=[f'Phase {phase}' for phase in pivot_mean_hit_rates.columns])
plt.xticks(rotation=0)
plt.show()


In [ ]:
pivot_mean_hit_rates

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the human experiment data from the CSV file
results_df = pd.read_csv('Model.csv')

frequency_order = [25, 50, 75]


results_df['Base Rate'] = pd.Categorical(results_df['Base Rate'], categories=frequency_order, ordered=True)

# Group by frequency and phase, avg hit rates>
mean_false_rates = results_df.groupby(['Base Rate', 'Phase'])['FARate'].mean().reset_index()

pivot_mean_false_rates = mean_false_rates.pivot_table(index='Base Rate', columns='Phase', values='FARate')

# Plotting
pivot_mean_false_rates.plot(kind='bar', figsize=(12, 8), colormap='viridis')
plt.title('Average False Alarm Rates for Each Frequency and Phase')
plt.xlabel('Frequency(Base_Rate)')
plt.ylabel('Average False Rate')
plt.legend(title='Phase', loc='upper right', labels=[f'Phase {phase}' for phase in pivot_mean_false_rates.columns])
plt.xticks(rotation=0)
plt.show()


In [ ]:
pivot_mean_false_rates

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the human experiment data from the CSV file
results_df = pd.read_csv('Model.csv')

frequency_order = [25, 50, 75]


results_df['Base Rate'] = pd.Categorical(results_df['Base Rate'], categories=frequency_order, ordered=True)

# Group by frequency and phase, avg hit rates>
mean_dprime_rates = results_df.groupby(['Base Rate', 'Phase'])['dprime'].mean().reset_index()

pivot_mean_dprime_rates = mean_dprime_rates.pivot_table(index='Base Rate', columns='Phase', values='dprime')

# Plotting
pivot_mean_dprime_rates.plot(kind='bar', figsize=(12, 8), colormap='viridis')
plt.title('Average D-prime for Each Frequency and Phase')
plt.xlabel('Frequency(Base_Rate)')
plt.ylabel('Average D-prime Rate')
plt.legend(title='Phase', loc='upper right', labels=[f'Phase {phase}' for phase in pivot_mean_dprime_rates.columns])
plt.xticks(rotation=0)
plt.show()


In [ ]:
pivot_mean_dprime_rates

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the human experiment data from the CSV file
results_df = pd.read_csv('Model.csv')

frequency_order = [25, 50, 75]


results_df['Base Rate'] = pd.Categorical(results_df['Base Rate'], categories=frequency_order, ordered=True)

# Group by frequency and phase, avg hit rates>
mean_bias_rates = results_df.groupby(['Base Rate', 'Phase'])['Bias'].mean().reset_index()

pivot_mean_bias_rates = mean_bias_rates.pivot_table(index='Base Rate', columns='Phase', values='Bias')

# Plotting
pivot_mean_bias_rates.plot(kind='bar', figsize=(12, 8), colormap='viridis')
plt.title('Average Bias for Each Frequency and Phase')
plt.xlabel('Frequency(Base_Rate)')
plt.ylabel('Average Bias Rate')
plt.legend(title='Phase', loc='upper right', labels=[f'Phase {phase}' for phase in pivot_mean_bias_rates.columns])
plt.xticks(rotation=0)
plt.show()


In [ ]:
pivot_mean_bias_rates